In [1]:
import sys
import os

# Path to the project root (one level above the notebooks folder)
project_root = os.path.abspath("..")

# Add to Python path if not already present
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added:", project_root)

Project root added: /home/zervaki/Thesis_New


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# MVPC (original)
from mvpc.mvpc_pipeline import MVPC
from mvpc.ci_tests.gauss_permc import gauss_ci_td, gauss_ci_permc
from mvpc.ci_tests.gauss_drw import gauss_ci_drw

# MVPC Oracle
from mvpc.mvpc_oracle import MVPC_Oracle

# Missingness generation (synthetic)
from data.synthetic_data_generation.missingness_synthetic import (
    create_mar_ind, create_mnar_ind,
    generate_missing_values, generate_mcar_reference
)

# DAG utilities
from data.synthetic_data_generation.dag_and_data import (
    detect_colliders, detect_collider_parents
)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")


In [3]:
df = pd.read_csv("../data/processed_mimic/processed_mimic_24h_labs_demographics.csv")
print(df.shape)

mvpc_vars = [
    "Sodium", "Potassium", "Chloride", "Creatinine", "Urea Nitrogen",
    "Hematocrit", "Hemoglobin", "WBC", "Platelet Count", "Glucose",
    "anchor_age", "length_of_stay_hours",
]

X_real = df[mvpc_vars].to_numpy()

scaler = StandardScaler()
X_complete = scaler.fit_transform(X_real)

print("X_complete shape:", X_complete.shape)


(234, 25)
X_complete shape: (234, 12)


In [4]:
num_var = len(mvpc_vars)
idx = {v: i for i, v in enumerate(mvpc_vars)}

adj_true = np.zeros((num_var, num_var))

# Collider 1: Creatinine ← Sodium, Potassium
adj_true[idx["Sodium"], idx["Creatinine"]] = 1
adj_true[idx["Potassium"], idx["Creatinine"]] = 1

# Collider 2: Urea Nitrogen ← Chloride, Potassium
adj_true[idx["Chloride"], idx["Urea Nitrogen"]] = 1
adj_true[idx["Potassium"], idx["Urea Nitrogen"]] = 1

# Collider 3: Glucose ← Sodium, Chloride
adj_true[idx["Sodium"], idx["Glucose"]] = 1
adj_true[idx["Chloride"], idx["Glucose"]] = 1

# Collider 4: Hemoglobin ← Creatinine, Glucose
adj_true[idx["Creatinine"], idx["Hemoglobin"]] = 1
adj_true[idx["Glucose"], idx["Hemoglobin"]] = 1

# Collider 5: WBC ← Glucose, Urea Nitrogen
adj_true[idx["Glucose"], idx["WBC"]] = 1
adj_true[idx["Urea Nitrogen"], idx["WBC"]] = 1

# Collider 6: Platelet Count ← Hemoglobin, Hematocrit
adj_true[idx["Hemoglobin"], idx["Platelet Count"]] = 1
adj_true[idx["Hematocrit"], idx["Platelet Count"]] = 1

# Collider 7: LOS ← Hemoglobin, WBC, anchor_age
adj_true[idx["Hemoglobin"], idx["length_of_stay_hours"]] = 1
adj_true[idx["WBC"], idx["length_of_stay_hours"]] = 1
adj_true[idx["anchor_age"], idx["length_of_stay_hours"]] = 1

adj_true


array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [5]:
# ---------------------------------------------------------
# Generate large synthetic dataset from your DAG
# ---------------------------------------------------------

def gen_linear_gaussian_data(adj, n_samples, seed=42, noise_scale=1.0):
    rng = np.random.default_rng(seed)
    d = adj.shape[0]
    X = np.zeros((n_samples, d))

    # Random weights for edges
    W = adj * rng.normal(loc=0.8, scale=0.3, size=adj.shape)

    # Topological order (0..d-1 works because your DAG is acyclic)
    order = list(range(d))

    for j in order:
        parents = np.where(adj[:, j] == 1)[0]
        if len(parents) == 0:
            X[:, j] = rng.normal(0, noise_scale, size=n_samples)
        else:
            X[:, j] = X[:, parents] @ W[parents, j] + rng.normal(0, noise_scale, size=n_samples)

    return X

# Choose a large sample size
n_large = 5000  # you can try 2000, 5000, 10000

X_complete = gen_linear_gaussian_data(adj_true, n_samples=n_large, seed=42)

print("Synthetic X_complete shape:", X_complete.shape)


Synthetic X_complete shape: (5000, 12)


In [6]:
colliders = detect_colliders(adj_true)
collider_parents = detect_collider_parents(adj_true, colliders)

print("Colliders:", colliders)
print("Collider parents:", collider_parents)


Colliders: [3, 4, 6, 7, 8, 9, 11]
Collider parents: [[np.int64(0), np.int64(1)], [np.int64(1), np.int64(2)], [np.int64(3), np.int64(9)], [np.int64(4), np.int64(9)], [np.int64(5), np.int64(6)], [np.int64(0), np.int64(2)], [np.int64(6), np.int64(7), np.int64(10)]]


In [7]:
seed = 42

ms_mar, prt_ms_mar = create_mar_ind(
    colliders, collider_parents, num_var,
    num_extra_e=3, num_m=6, seed=seed
)

ms_mnar, prt_ms_mnar = create_mnar_ind(
    colliders, collider_parents, num_var,
    num_extra_e=3, num_m=6, seed=seed
)


In [8]:
def build_prt_m_from_ms(ms, prt_ms):
    prt_dict = {}
    for m, p in zip(ms, prt_ms):
        prt_dict.setdefault(m, []).append(p)
    m_inds = sorted(prt_dict.keys())
    return {"m": m_inds, "prt": prt_dict}


In [9]:
X_mar = generate_missing_values(
    X_complete=X_complete,
    ms=ms_mar,
    prt_ms=prt_ms_mar,
    p_missing_h=0.9,
    p_missing_l=0.1,
    seed=seed,
)

X_mcar = generate_mcar_reference(
    X_complete=X_complete,
    X_mar=X_mar,
    ms=ms_mar,
    seed=seed,
)

X_mnar = generate_missing_values(
    X_complete=X_complete,
    ms=ms_mnar,
    prt_ms=prt_ms_mnar,
    p_missing_h=0.9,
    p_missing_l=0.1,
    seed=seed,
)


In [10]:
prt_m_mar  = build_prt_m_from_ms(ms_mar,  prt_ms_mar)
prt_m_mnar = build_prt_m_from_ms(ms_mnar, prt_ms_mnar)
prt_m_mcar = prt_m_mar  # MCAR uses same structure, permuted mask


In [11]:
def shd_directed(G_est, G_true):
    return np.sum(G_est != G_true)

def f1_directed(G_est, G_true):
    est = G_est.flatten()
    true = G_true.flatten()

    TP = np.sum((est == 1) & (true == 1))
    FP = np.sum((est == 1) & (true == 0))
    FN = np.sum((est == 0) & (true == 1))

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall    = TP / (TP + FN) if TP + FN > 0 else 0
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)


In [ ]:
def record_result(results_list, mode, method, n, shd, f1):
    results_list.append({
        "mode": mode,
        "method": method,
        "n": n,
        "shd": shd,
        "f1": f1,
    })


In [ ]:
# Experiment configuration
sample_sizes = [500, 1000, 2000, 5000, 10000, 50000]
n_reps = 15

num_var = 20
num_extra_e = 3
num_m = 6

p_missing_h = 0.9
p_missing_l = 0.1

modes = ["mar", "mnar"]
methods = ["td", "permc", "drw"]

all_results = []


In [ ]:
from mvpc.mvpc_oracle import MVPC_Oracle

for n in sample_sizes:
    for rep in range(n_reps):

        # 1. Generate a random DAG with collider structure
        # (You can reuse your existing DAG generator or plug in your own)
        # For now, we reuse your adj_true if you prefer consistency:
        adj = adj_true.copy()

        # 2. Generate synthetic data
        X_complete = gen_linear_gaussian_data(adj, n_samples=n, seed=rep)

        # 3. Detect colliders
        colliders = detect_colliders(adj)
        collider_parents = detect_collider_parents(adj, colliders)

        # 4. Generate missingness structures
        ms_mar, prt_ms_mar = create_mar_ind(
            colliders, collider_parents, num_var,
            num_extra_e=num_extra_e, num_m=num_m, seed=rep
        )
        ms_mnar, prt_ms_mnar = create_mnar_ind(
            colliders, collider_parents, num_var,
            num_extra_e=num_extra_e, num_m=num_m, seed=rep
        )

        # 5. Build oracle prt_m
        prt_m_mar  = build_prt_m_from_ms(ms_mar,  prt_ms_mar)
        prt_m_mnar = build_prt_m_from_ms(ms_mnar, prt_ms_mnar)

        # 6. Generate missing data
        X_mar = generate_missing_values(
            X_complete, ms_mar, prt_ms_mar,
            p_missing_h=p_missing_h, p_missing_l=p_missing_l, seed=rep
        )
        X_mnar = generate_missing_values(
            X_complete, ms_mnar, prt_ms_mnar,
            p_missing_h=p_missing_h, p_missing_l=p_missing_l, seed=rep
        )

        # 7. Run MVPC Oracle for each mode
        for mode, X_m, prt_m in [
            ("MAR",  X_mar,  prt_m_mar),
            ("MNAR", X_mnar, prt_m_mnar),
        ]:

            for method in methods:
                if method == "td":
                    mvpc = MVPC_Oracle(gauss_ci_td, gauss_ci_td)
                elif method == "permc":
                    mvpc = MVPC_Oracle(gauss_ci_td, gauss_ci_permc)
                elif method == "drw":
                    mvpc = MVPC_Oracle(gauss_ci_td, gauss_ci_drw)

                G = mvpc.run(X_m, prt_m)["G_corrected"]

                shd = shd_directed(G, adj)
                f1  = f1_directed(G, adj)

                record_result(all_results, mode, method, n, shd, f1)


In [12]:
def run_mvpc_oracle_all_methods(X_m, adj_true, prt_m):
    results = {}

    mvpc_td = MVPC_Oracle(indep_test=gauss_ci_td, corr_test=gauss_ci_td)
    G_td = mvpc_td.run(X_m, prt_m)["G_corrected"]
    results["TD"] = {"shd": shd_directed(G_td, adj_true), "f1": f1_directed(G_td, adj_true)}

    mvpc_permc = MVPC_Oracle(indep_test=gauss_ci_td, corr_test=gauss_ci_permc)
    G_permc = mvpc_permc.run(X_m, prt_m)["G_corrected"]
    results["PERMC"] = {"shd": shd_directed(G_permc, adj_true), "f1": f1_directed(G_permc, adj_true)}

    mvpc_drw = MVPC_Oracle(indep_test=gauss_ci_td, corr_test=gauss_ci_drw)
    G_drw = mvpc_drw.run(X_m, prt_m)["G_corrected"]
    results["DRW"] = {"shd": shd_directed(G_drw, adj_true), "f1": f1_directed(G_drw, adj_true)}

    return results


In [13]:
results_oracle = {
    "MAR":  run_mvpc_oracle_all_methods(X_mar,  adj_true, prt_m_mar),
    "MNAR": run_mvpc_oracle_all_methods(X_mnar, adj_true, prt_m_mnar),
    "MCAR": run_mvpc_oracle_all_methods(X_mcar, adj_true, prt_m_mcar),
}

results_oracle


{'MAR': {'TD': {'shd': np.int64(19), 'f1': np.float64(0.5957446808510638)},
  'PERMC': {'shd': np.int64(17), 'f1': np.float64(0.6222222222222223)},
  'DRW': {'shd': np.int64(19), 'f1': np.float64(0.5957446808510638)}},
 'MNAR': {'TD': {'shd': np.int64(19), 'f1': np.float64(0.5957446808510638)},
  'PERMC': {'shd': np.int64(17), 'f1': np.float64(0.6222222222222223)},
  'DRW': {'shd': np.int64(19), 'f1': np.float64(0.5957446808510638)}},
 'MCAR': {'TD': {'shd': np.int64(19), 'f1': np.float64(0.5777777777777778)},
  'PERMC': {'shd': np.int64(17), 'f1': np.float64(0.6046511627906976)},
  'DRW': {'shd': np.int64(19), 'f1': np.float64(0.5777777777777778)}}}

In [14]:
rows = []
for mode, methods in results_oracle.items():
    for method, metrics in methods.items():
        rows.append({
            "mode": mode,
            "method": method,
            "shd": float(metrics["shd"]),
            "f1": float(metrics["f1"]),
        })

df_oracle_results = pd.DataFrame(rows)
df_oracle_results


,mode,method,shd,f1
0,MAR,TD,19.0,0.595745
1,MAR,PERMC,17.0,0.622222
2,MAR,DRW,19.0,0.595745
3,MNAR,TD,19.0,0.595745
4,MNAR,PERMC,17.0,0.622222
5,MNAR,DRW,19.0,0.595745
6,MCAR,TD,19.0,0.577778
7,MCAR,PERMC,17.0,0.604651
8,MCAR,DRW,19.0,0.577778


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

for mode in ["MAR", "MNAR"]:
    plt.figure(figsize=(7, 5))
    sub = summary[summary["mode"] == mode]

    sns.lineplot(
        data=sub,
        x="n",
        y="mean_shd",
        hue="method",
        marker="o"
    )

    plt.title(f"Skeleton SHD vs sample size ({mode})")
    plt.xlabel("Sample size (n)")
    plt.ylabel("Mean SHD")
    plt.legend(title="Method")
    plt.xscale("log")  # optional but nice for 500 → 50000
    plt.tight_layout()
    plt.show()


NameError: name 'summary' is not defined

<Figure size 700x500 with 0 Axes>

In [ ]:
for mode in ["MAR", "MNAR"]:
    plt.figure(figsize=(7, 5))
    sub = summary[summary["mode"] == mode]

    sns.lineplot(
        data=sub,
        x="n",
        y="mean_f1",
        hue="method",
        marker="o"
    )

    plt.title(f"Skeleton F1 vs sample size ({mode})")
    plt.xlabel("Sample size (n)")
    plt.ylabel("Mean F1 score")
    plt.legend(title="Method")
    plt.xscale("log")
    plt.tight_layout()
    plt.show()
